<a href="https://colab.research.google.com/github/Kither12/CS114.O11-22521494/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install jiwer
!pip install gradio==3.50

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import Wav2Vec2CTCTokenizer
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [ ]:
from transformers import Wav2Vec2Processor
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer
model = AutoModelForCTC.from_pretrained("/content/drive/MyDrive/checkpoint-7000")

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
import gradio as gr
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2ProcessorWithLM
import torch
import torchaudio


In [ ]:
def confidence_score(word_dict, index):
    probs = pred_scores[index, word_dict["start_offset"]: word_dict["end_offset"]]
    return round(torch.sum(probs).item() / (len(probs)), 4)

In [ ]:
def inference(audio):
    speech_array, sampling_rate = torchaudio.load(audio)
    speech_array = processor(speech_array, sampling_rate=16000)
    with torch.no_grad():
      logits = model(torch.tensor(speech_array['input_values'][0])).logits
    pred_ids = torch.argmax(logits, dim=-1)

    scores = torch.nn.functional.log_softmax(logits, dim=-1)
    pred_scores = scores.gather(1, pred_ids.unsqueeze(-1))[:, :, 0]

    output = processor.batch_decode(pred_ids, output_word_offsets=True)

    # add confidence
    def confidence_score(word_dict, index):
        probs = pred_scores[index, word_dict["start_offset"]: word_dict["end_offset"]]
        return round(torch.sum(probs).item() / (len(probs)), 4)

    confidence_scores = []
    for i in range(1):
        confidence_scores.append({d["word"]: confidence_score(d, i) + 100 for d in output.word_offsets[i]})
    print(f'confidence scores {confidence_scores}')
    print(f'model predict {processor.batch_decode(pred_ids)[0]}')
    return


inputs = gr.inputs.Audio(label="Input Audio", type="filepath")
outputs = gr.outputs.Textbox(label="Output Text")
title = 'wav2vec2'
description = f"Gradio demo for a {title}. To use it, simply upload your audio, or click one of the examples to load them. Read more at the links below. Currently supports .wav 16_000hz files"
article = "<p style='text-align: center'><a href='https://github.com/egorsmkv/wav2vec2-uk-demo' target='_blank'> Github repo</a> | <a href='<HF Space link>' target='_blank'>Pretrained model</a> | Made with help from <a href='https://github.com/robinhad' target='_blank'>@robinhad</a></p>"
examples = []
gr.Interface(
    inference,
    inputs,
    outputs,
    title=title,
    description=description,
    article=article,
    examples=examples,
).launch(debug=True)

<ipython-input-40-388ca2085b0f>:26: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs = gr.inputs.Audio(label="Input Audio", type="filepath")
<ipython-input-40-388ca2085b0f>:26: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs = gr.inputs.Audio(label="Input Audio", type="filepath")
<ipython-input-40-388ca2085b0f>:27: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs = gr.outputs.Textbox(label="Output Text")


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://c9847ae6ffdab8b103.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


confidence scores [{"life's": 89.5671, 'togh': 89.6137, 'but': 92.1863, "it's": 90.6489, 'tofer': 90.1736, 'whene': 88.1702, 'you': 92.2766, 'stuped': 89.3639}]
model predict life's togh but it's tofer whene you stuped
